<h1>Hello to Flex Matcher by LLM MultiAgent</h1>

In [4]:
!pip install flexmatcher

  Using cached flexmatcher-1.0.4-py2.py3-none-any.whl (13 kB)
     --------------------------------------- 34.1/34.1 MB 12.8 MB/s eta 0:00:00
     ---------------------------------------- 7.1/7.1 MB 14.2 MB/s eta 0:00:00
  Using cached python-Levenshtein-0.12.0.tar.gz (48 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
     -------------------------------------- 302.2/302.2 kB 6.2 MB/s eta 0:00:00
  Created wheel for python-Levenshtein: filename=python_Levenshtein-0.12.0-cp37-cp37m-win_amd64.whl size=79527 sha256=43d26acdfc4cb720a0ee942ac416144e55b6c38fe10f11cb6308e5788c179c44
  Stored in directory: c:\users\matte\appdata\local\pip\cache\wheels\45\30\6d\09532ae6e0de87a51278474711f6c089785678a24d772ba4b6
Successfully built python-Levenshtein


Reason for being yanked: Insecure, upgrade to 0.12.1


In [2]:
from flexmatcher import FlexMatcher
import pandas as pd
import numpy as np
import os
import json


print(np.__version__)
print(pd.__version__)

#####
#la libreria flexmatcher fa un po schifo e funziona solo con vecchie versioni di pandas

1.16.6
0.25.3


<h3>Questo serve a farsi stampare il tipo di codifica del csv... se serve</h3>

In [28]:
!pip install chardet
import chardet

file_path = r"sources\output_govuk_bigsize.json"
with open(file_path, "rb") as f:
    result = chardet.detect(f.read(1000000))
    print("Codifica rilevata:", result['encoding'])


Codifica rilevata: ascii


<h3>Questi sono i 4 file da cui impara con il mapping dato</h3>

In [4]:
file_path = r"../sources/AmbitionBox.csv"
file_path2 = r"../sources/DDD-cbinsight.com.csv"
file_path3 = r"../sources/output_globaldata.json"
file_path4 = r"../sources/output_govuk_bigsize.json"

df1 = pd.read_csv(file_path, encoding='Windows-1252')
df2 = pd.read_csv(file_path2, encoding='ascii')
#df2 = df2.drop(df2.columns[0], axis=1)
df3 = pd.read_json(file_path3)
df4 = pd.read_json(file_path4)

df1 = df1.applymap(str)
df2 = df2.applymap(str)
df3 = df3.applymap(str)
df4 = df4.applymap(str)

<h3 class=color: red>Questa é la mappa su cui si basa per predirre mapping<h3>

In [5]:
#(name,category,address ,city, country, anno fondazione, link(pagina http o www), rank, 
# employees, market_cap, ceo, revenue, investors,type(sector(?)), postalCode, IBAN,sic_code, valuation)

#(rank, ceo, postalCode, IBAN,sic_code) questi direi che li piscio alla grande
#type -> output_govuk_bigsize.json
#rank, ceo  -> valueToday.jsonl

#AmbitionBox.csv
data1_mapping = {'Name': 'name', 
                 'Industry': 'category',
                 'Headquarter': 'city',
                 'Ownership': 'ownership',
                 'Foundation Year': 'fundation_year'}
#DDD-cbinsight.com.csv
data2_mapping = {'Unnamed: 0': 'unnamed',
                 'name': 'name',
                 'valuation': 'valuation',
                 'dateJoined': 'fundation_year',
                 'country': 'country',
                 'city': 'city',
                 'industry': 'category',
                 'investors': 'investors',
                 'founded': 'founded',
                 'stage': 'stage',
                 'totalRaised': 'total_raised'}
#output_globaldata.json
data3_mapping = {'name': 'name',
                 'headquarters': 'city',
                 'number_of_employees': 'employees',
                 'address': 'address',
                 'industry': 'category',
                 'website': 'link',
                 'market_cap': 'market_cap',
                 'telephone': 'telephone',
                 'revenue': 'revenue'}
#output_global_govuk_bigsize.jsonl
data4_mapping = {'id': 'unnamed',
                 'name': 'name',
                 'world_rank': 'rank',
                 'annual_revenue_in_usd': 'revenue',
                 'headquarters_region_city': 'city',
                 'headquarters_country': 'country',
                 'company_website': 'link',
                 'number_of_employees': 'employees',
                 'ceo': 'ceo',
                 'company_business': 'category',
                 'company_number': 'unnamed',
                 'registered_office_address': 'address',
                 'company_status': 'satus',
                 'company_type': 'type',
                 'company_creation_date': 'fundation_year',
                 'nature_of_business': 'category'}

<h1>TRAIN</h1>

In [6]:
schema_list = [df1, df2, df3, df4]
mapping_list = [data1_mapping, data2_mapping, data3_mapping, data4_mapping]

fm = FlexMatcher(schema_list, mapping_list, sample_size=100)
fm.train()
fm.save_model("flexmatcher_model_companies.pkl")


Create training data ...
Training FlexMatcher ...


c:\Users\matte\anaconda3\envs\flexmatcher_env\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
c:\Users\matte\anaconda3\envs\flexmatcher_env\lib\site-packages\sklearn\linear_model\_logistic.py:940: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/sta

Training Completed ...


<h1>Mo proviamo a usare il FlexMatcher addestrato per unire tutto in una grande tabella</h1>

In [32]:
!pip install xlrd

     ---------------------------------------- 96.5/96.5 kB 1.8 MB/s eta 0:00:00


In [8]:
SRC = "../sources"
ENCODINGS = ["utf-8", "ISO-8859-1", "latin1", "Windows-1252", "ascii"]

schemas = []
dfs = []

for filename in os.listdir(SRC):
    if filename.startswith("wissel"):
        continue 

    file_path = os.path.join(SRC, filename)
    df = None

    if file_path.endswith('.csv'):
        for enc in ENCODINGS:
            try:
                df = pd.read_csv(file_path, encoding=enc)
                break
            except Exception:
                continue

    elif file_path.endswith('.json'):
        with open(file_path, 'r', encoding='utf-8') as f:
            data = json.load(f)
        df = pd.io.json.json_normalize(data) if isinstance(data, list) else pd.io.json.json_normalize([data])

    elif file_path.endswith('.jsonl'):
        df = pd.read_json(file_path, lines=True, encoding='utf-8')

    elif file_path.endswith(('.xls', '.xlsx')):
        df = pd.read_excel(file_path)

    if df is not None:
        df = df.applymap(str)
        predicted_mapping = fm.make_prediction(df)
        df_renamed = df.rename(columns=predicted_mapping)
        dfs.append(df_renamed)

df_all = pd.concat(dfs, ignore_index=True)
df_all.to_csv("merged_companies.csv", index=False)
print(df_all.head())

c:\Users\matte\anaconda3\envs\flexmatcher_env\lib\site-packages\flexmatcher\classify\charDistClassifier.py:121: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  features = feat_df.ix[:, 1:].as_matrix()
c:\Users\matte\anaconda3\envs\flexmatcher_env\lib\site-packages\pandas\core\indexing.py:822: FutureWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#ix-indexer-is-deprecated
  retval = getattr(retval, self.name)._getitem_axis(key, axis=i)
c:\Users\matte\anaconda3\envs\flexmatcher_env\lib\site-packages\flexmatcher\classify\charDistClassifier.py:121: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  features = feat_df

  address                  category  \
0     NaN  IT Services & Consulting   
1     NaN  IT Services & Consulting   
2     NaN  IT Services & Consulting   
3     NaN                   Banking   
4     NaN                   Banking   

                                        city country employees founded  \
0                 Mumbai, Maharashtra, India     NaN       NaN     NaN   
1                            Dublin, Ireland     NaN       NaN     NaN   
2  Teaneck. New Jersey., United States (USA)     NaN       NaN     NaN   
3                 Mumbai, Maharashtra, India     NaN       NaN     NaN   
4                 Mumbai, Maharashtra, India     NaN       NaN     NaN   

      fundation_year investors link market_cap        name ownership revenue  \
0  1968 (55 yrs old)       NaN  NaN        NaN         TCS    Public     NaN   
1  1989 (34 yrs old)       NaN  NaN        NaN   Accenture    Public     NaN   
2  1994 (29 yrs old)       NaN  NaN        NaN   Cognizant   Private     NaN   


<h1>Mo andrebbe fatto per i file wissel che sono dei dipendenti</h1>
<h3>il punto é metti tutto insieme o separati? In ogni caso dovrebbe essere piuttosto semplice</h3>